In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
from pathlib import Path

In [4]:
datasetpath = Path("./private/data")
filepath = datasetpath/"geometrics_pat_clean.feather"
data = pd.read_feather(filepath)

In [5]:
data.describe()

,index,endo,epi,volume,longitudinal_distance_epi,longitudinal_distance_endo,longitudinal_distance,radial_distance_epi,radial_distance_endo,radial_distance,...,radial_distance-b,wall_thickness-b,global_longitudinal_length_epi-b,global_longitudinal_length_endo-b,global_longitudinal_length-b,global_circumferential_length_epi-b,global_circumferential_length_endo-b,global_circumferential_length-b,gamma-b,pressure-b
count,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000,...,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000,722172.000000
mean,217.208452,60.206987,-60.275718,0.647480,83.530689,73.626516,78.578603,30.185387,13.831594,22.008491,...,16.331929,34.709216,17.019189,16.636404,16.761809,16.597510,16.276441,16.324122,33.995954,22.929171
std,126.986264,12.100817,12.076226,0.171593,4.928667,4.743545,4.813071,0.807611,1.924450,1.314526,...,9.234805,9.171212,10.426227,9.786458,9.997710,9.614765,9.144922,9.222517,10.126123,14.959494
min,0.000000,40.000000,-80.000000,0.400717,72.854540,63.821816,68.511640,27.198589,9.653598,18.452143,...,1.000000,14.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,107.000000,50.000000,-70.000000,0.488593,79.945185,69.963192,74.996594,29.635574,12.120928,20.919241,...,8.000000,27.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,26.000000,9.000000
50%,216.000000,60.000000,-60.000000,0.606488,83.345319,73.323815,78.305148,30.225879,13.588827,21.857512,...,16.000000,35.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,34.000000,22.000000
75%,327.000000,70.000000,-50.000000,0.790801,86.841054,76.848549,81.830530,30.754085,15.477431,23.081910,...,24.000000,43.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,42.000000,36.000000
max,440.000000,80.000000,-40.000000,0.999997,101.129277,90.520248,95.824758,32.159797,17.999854,25.078146,...,36.000000,50.000000,50.000000,49.000000,50.000000,39.000000,34.000000,36.000000,50.000000,50.000000


In [6]:
assert len(data.columns.to_series()[np.isinf(data).any()]) == 0
assert data.isnull().values.any() == False
assert data.isin([np.inf, -np.inf]).values.any() == False

In [7]:
train, test = train_test_split(data, test_size=0.4, random_state=42) 
valid, test = train_test_split(test, test_size=0.4, random_state=42) 

In [8]:
print("Train: {:.3f}, Validation: {:.3f}, Test:{:.3f}".format(
    len(train)/len(data), len(valid)/len(data), len(test)/len(data) ))

Train: 0.600, Validation: 0.240, Test:0.160


In [9]:
def split_files(df, path, mode, n=100):

  s_space = np.linspace(0,df.count().max(), n, dtype=int, endpoint=True)

  if not os.path.exists(os.path.join(path, mode)):
    os.makedirs(os.path.join(path, mode))

  for i, (iL, iR) in enumerate(zip(s_space[:], s_space[1:])):
    elems = df.iloc[iL:iR]
    elems = elems.sample(frac=1).reset_index(drop=True)
    elems.to_csv("{}/{}/{}.csv".format(path, mode, (int(i))), index=False)

In [10]:
split_files(train, datasetpath, "geometrics_pat_clean/train")
split_files(valid, datasetpath, "geometrics_pat_clean/valid")
split_files(test, datasetpath,  "geometrics_pat_clean/test")